In [2]:
import pandas as pd
import numpy as np
import calendar
from time import strptime
from datetime import datetime
import requests
from matplotlib import pyplot as plt

from toby_config import toby_api_key

In [44]:
# Installing bokeh in order to plot on top of a map

import bokeh.io
from bokeh.models import (
  GMapPlot, GMapOptions, ColumnDataSource, Circle, LogColorMapper, BasicTicker, ColorBar,
    DataRange1d, PanTool, WheelZoomTool, Range1d, Annotation, Legend, LegendItem, GlyphRenderer)
    
from bokeh.plotting import figure, show, output_file, output_notebook

In [3]:
df= pd.read_csv("crime.csv")
df['semi_year'] = df['REPORTED_DATE'].map(lambda x: x)
df['REPORTED_DATE'] = pd.to_datetime(df['REPORTED_DATE'])
df['year'] = df['REPORTED_DATE'].map(lambda x: str(x.year) +'A' if x.month < 7 else str(x.year) +"B")
df= df[df['IS_TRAFFIC'] == 0].copy()
df.head()

,INCIDENT_ID,OFFENSE_ID,OFFENSE_CODE,OFFENSE_CODE_EXTENSION,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,LAST_OCCURRENCE_DATE,REPORTED_DATE,INCIDENT_ADDRESS,...,GEO_LON,GEO_LAT,DISTRICT_ID,PRECINCT_ID,NEIGHBORHOOD_ID,IS_CRIME,IS_TRAFFIC,Year,semi_year,year
0,2.016377e+09,2.020000e+15,5213,0,weapon-unlawful-discharge-of,all-other-crimes,6/15/2016 23:31,NaN,2016-06-15,NaN,...,-104.809881,39.773188,5,521,montbello,1,0,2016,6/15/2016,2016A
1,2.018600e+10,2.020000e+16,2399,0,theft-other,larceny,10/11/2017 12:30,10/11/2017 16:55,2018-01-29,NaN,...,-104.781434,39.785649,5,522,gateway-green-valley-ranch,1,0,2018,1/29/2018,2018A
2,2.016600e+10,2.020000e+16,2305,0,theft-items-from-vehicle,theft-from-motor-vehicle,3/4/2016 20:00,4/25/2016 8:00,2016-04-26,2932 S JOSEPHINE ST,...,-104.957381,39.663490,3,314,wellshire,1,0,2016,4/26/2016,2016A
3,2.018723e+08,2.020000e+14,2399,0,theft-other,larceny,1/30/2018 19:20,NaN,2018-01-30,705 S COLORADO BLVD,...,-104.941440,39.702698,3,312,belcaro,1,0,2018,1/30/2018,2018A
4,2.017411e+09,2.020000e+15,2303,0,theft-shoplift,larceny,6/22/2017 20:53,NaN,2017-06-23,2810 E 1ST AVE,...,-104.955370,39.717107,3,311,cherry-creek,1,0,2017,6/23/2017,2017A


In [4]:
def get_crime_type_locations(crime_type, year): #this function sorts data by crime type and year, outputs lat & lng
    year_filter = df[df['year'] == year].copy()
    crime_filter = year_filter[year_filter['OFFENSE_CATEGORY_ID'] == crime_type].copy()
    
    lat = crime_filter['GEO_LAT'].dropna()
    lat = lat[lat > 38].copy()
    lat = lat[lat < 41].copy()

    lng = crime_filter['GEO_LON'].dropna()
    lng = lng[lng > -105.5].copy()
    lng = lng[lng < -104.5].copy()
    return lat, lng

In [5]:
# plot is our figure, this allows us to plot multiple datasets on same map
def map_plot(plot, lat_data, lng_data, color_in): 
    color_list = [color_in for each in lat_data]
    source = ColumnDataSource(
        data=dict(
            lat= lat_data,
            lon= lng_data,
            size= 1.5*np.ones(len(lat_data)),
            color= color_list
        )
    )
    circle = Circle(x="lon", y="lat", size="size", fill_color='color', fill_alpha=0.5, line_color=None)
    plot.add_glyph(source, circle)


# Main Code to Run

In [77]:
#year_list = df['year'].unique()
year_list= ['2018A','2017A']
crime_list = df['OFFENSE_CATEGORY_ID'].unique()
color_list = ['deeppink','red','orange', 
              'yellow','black','aqua','teal',
              'dodgerblue','mediumblue','darkviolet',
              'purple', 'white','gray','greenyellow']

for year in year_list:
    map_options = GMapOptions(lat=39.738711, lng=-104.989411, map_type="roadmap", zoom=11)
    plot = GMapPlot(
        x_range= Range1d(), y_range= Range1d(), map_options= map_options
    )
    plot.api_key = toby_api_key
    
    for i in range(len(crime_list)):
        lat, lng = get_crime_type_locations(crime_list[i], year)
        map_plot(plot, lat, lng, color_list[i])
    
    plot.add_tools(WheelZoomTool())
    plot.title.text = f"Crime in the Year {year}"
    show(plot) 


ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: color [renderer: GlyphRenderer(id='3d780517-4b65-433c-bd43-d211a68b81f0', ...)]
ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: color [renderer: GlyphRenderer(id='0d104114-10f4-4366-93c3-36001b178d7e', ...)]
ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: color [renderer: GlyphRenderer(id='cd6fd5c1-a01a-41c2-afa8-488b60957c7c', ...)]
ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: color [renderer: GlyphRenderer(id='3d780517-4b65-433c-bd43-d211a68b81f0', ...)]
ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: color [renderer: GlyphRenderer(id='0d104114-10f4-4366-93c3-36001b178d7e', ...)]
ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column 

## Mapping Experiment

In [25]:
lat_2018A = df[df['year'] == '2018A']['GEO_LAT'].dropna()
lat_2018A = lat_2018A[lat_2018A > 38].copy()
lat_2018A = lat_2018A[lat_2018A < 41].copy()

lng_2018A = df[df['year'] == '2018A']['GEO_LON'].dropna()
lng_2018A = lng_2018A[lng_2018A_place > -105.5].copy()
lng_2018A = lng_2018A[lng_2018A < -104].copy()

lat_2017A = df[df['year'] == '2017A']['GEO_LAT'].dropna()
lat_2017A = lat_2017A[lat_2017A > 38].copy()
lat_2017A = lat_2017A[lat_2017A < 41].copy()

lng_2017A = df[df['year'] == '2017A']['GEO_LON'].dropna()
lng_2017A = lng_2017A[lng_2017A > -105.5].copy()
lng_2017A = lng_2017A[lng_2017A < -104].copy()

lng_2018A.head()


1   -104.781434
3   -104.941440
5   -104.961928
6   -105.025543
7   -104.983794
Name: GEO_LON, dtype: float64

In [56]:
map_options = GMapOptions(lat=39.738711, lng=-104.989411, map_type="roadmap", zoom=11)
plot = GMapPlot(
    x_range= Range1d(), y_range= Range1d(), map_options=map_options
)
plot.title.text = "Hey look! It's a scatter plot on a map!"


color_blue = ['blue' for each in lat_2018A]
legend_thang = [LegendItem(label= '2018A') for each in lat_2018A]
source = ColumnDataSource(
    data=dict(
        lat= lat_2018A,
        lon= lng_2018A,
        size= 1.5*np.ones(len(lat_2018A)),
        color = color_blue,
        legend = Legend(items = legend_thang, click_policy = 'hide')
    
    )
)
circle = Circle(x="lon", y="lat", size="size", fill_color= 'color', fill_alpha=0.5, line_color=None)
plot.add_glyph(source, circle)

color_yellow = ['yellow' for each in lat_2017A]
legend_thang = [LegendItem(label= '2017A') for each in lat_2017A]
source2 = ColumnDataSource(
    data=dict(
        lat= lat_2017A,
        lon= lng_2017A,
        size= 1.5*np.ones(len(lat_2017A)),
        color = color_yellow,
        legend = Legend(items = legend_thang), click_policy = 'hide'
    )
)
circle2 = Circle(x="lon", y="lat", size="size", fill_color= 'color', fill_alpha=0.5, line_color=None)
plot.add_glyph(source2, circle2)

plot.api_key = toby_api_key
#p.legend.location = "top_left"
#p.legend.click_policy="hide"

show(plot)

ValueError: expected an element of ColumnData(String, Seq(Any)), got {'lat': 1         39.785649
3         39.702698
5         39.743149
6         39.732790
7         39.723423
9         39.757627
10        39.678463
11        39.694578
17        39.740117
18        39.701835
20        39.772489
23        39.783741
24        39.675747
25        39.652260
26        39.858704
27        39.687462
30        39.657232
32        39.735689
33        39.831895
34        39.729096
37        39.795550
38        39.775182
39        39.740226
40        39.750827
41        39.824058
42        39.784667
46        39.725138
51        39.862877
52        39.717051
53        39.682426
            ...    
461926    39.628867
461927    39.655634
461928    39.780208
461929    39.778046
461930    39.794470
461931    39.740293
461932    39.724702
461934    39.657011
461936    39.740352
461937    39.757564
461954    39.757844
461955    39.718903
461956    39.761632
465894    39.794550
467277    39.753790
467

In [ ]:
import gmaps

gmaps.configure(api_key= toby_api_key)

df_2018A = pd.DataFrame(columns = ['latitude','longitude'] )
df_2018A['latitude'] = lat_2018A
df_2018A['longitude'] = lng_2018A

df_2017A = pd.DataFrame(columns = ['latitude','longitude'] )
df_2017A['latitude'] = lat_2017A
df_2017A['longitude'] = lng_2017A




layer_2017A = gmaps.symbol_layer(
    df_2017A, fill_color='green', stroke_color='green', scale=2
 )
layer_2018A = gmaps.symbol_layer(
    df_2018A, fill_color='red', stroke_color='red', scale=2
 )

fig = gmaps.figure()
fig.add_layer(layer_2018A)
fig.add_layer(layer_2017A)
fig


